# Stemming
## What is?
**Stemming** is the process of reducing words to their root or base form, known as the *stem*. This technique is widely used in Natural Language Processing (NLP) to normalize words by removing suffixes, prefixes, or other morphological variations.

For example:
- The words "running," "runner," and "ran" can be reduced to the root "run."

The goal of stemming is to treat words with similar meanings as equivalent, which is useful for tasks like information retrieval, text analysis, and classification.

However, stemming may produce non-existent words in the language, as it does not consider grammatical context, unlike lemmatization.

## What for?
As this work aims to match Tasks and Objects from patents, work with post-processed stemming text can result in more matches.

In [1]:
!pip install pandas
!python -m spacy download pt_core_news_lg  # The large core for portuguese


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 7.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [6]:
import pandas as pd
import string
import spacy
import unicodedata

The raw patents can be found at `data/raw/patentes_inpi.csv` and the Task + Object database can be found at `data/raw/Base efeitos físicos publicada.xlsx`

In [8]:
df = pd.read_csv("../../data/raw/patentes_inpi.csv")
df_match = pd.read_excel("../../data/raw/Base efeitos físicos publicada.xlsx")

df.head()

,id_pedido,data_deposito,titulo,ipc,url,resumo,classifica_ipc
0,BR 11 2021 018393 0,02/03/2020,TRATAMENTO DE COLISÕES EM UPLINK,H04L 1/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,"A presente invenção se refere a métodos, sis...",H04L 1/18
1,BR 11 2021 018071 0,02/03/2020,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,H01T 13/14,https://busca.inpi.gov.br/pePI/servlet/Patente...,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,H01T 13/14 ; H01T 13/20 ; H01T 13/32 ; H0...
2,BR 11 2021 016947 4,02/03/2020,ANTICORPOS QUE RECONHECEM TAU,C07K 16/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,C07K 16/18 ; G01N 33/68
3,BR 10 2020 004169 0,02/03/2020,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,F24H 3/00,https://busca.inpi.gov.br/pePI/servlet/Patente...,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,F24H 3/008 ; F24H 4/06
4,BR 11 2021 006234 3,02/03/2020,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,C12N 15/10,https://busca.inpi.gov.br/pePI/servlet/Patente...,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,C12N 15/10


# spaCy
**spaCy** is a powerful and efficient Natural Language Processing (NLP) library in Python. It is designed for production use and provides tools for tasks such as tokenization, lemmatization, part-of-speech tagging, named entity recognition, dependency parsing, and more. spaCy is widely used in both research and industry due to its speed and ease of use.

## Key Features of spaCy
- **Tokenization**: Breaks text into individual words, punctuation, and symbols.
- **Lemmatization**: Reduces words to their base or dictionary form.
- **Part-of-Speech Tagging**: Identifies the grammatical role of each word in a sentence.
- **Named Entity Recognition (NER)**: Detects entities like names, dates, and locations in text.
- **Dependency Parsing**: Analyzes the grammatical structure of sentences.
- **Pre-trained Models**: Provides pre-trained language models for various languages.

## spaCy Language Cores

spaCy provides pre-trained language models, also known as **cores**, which are optimized for specific languages. These cores include linguistic annotations, word vectors, and statistical models for tasks like NER and dependency parsing.

### Examples of Language Cores
- **`en_core_web_sm`**: English small model.
- **`pt_core_news_lg`**: Portuguese large model.
- **`es_core_news_md`**: Spanish medium model.

### Choosing a Core
The size of the core determines its capabilities:
- **Small (`_sm`)**: Lightweight and fast, suitable for basic tasks.
- **Medium (`_md`)**: Includes word vectors for better accuracy.
- **Large (`_lg`)**: Provides the highest accuracy with extensive word vectors.

In [9]:
nlp = spacy.load("pt_core_news_lg")


def lemmatizer_spacy(text: str):
    """
    Lemmatizes the input text using spaCy.

    This function processes the input text with a spaCy language model to extract 
    the lemmatized (base) form of each word. It excludes stop words and punctuation 
    from the output.

    Args:
        text (str): The input text to be lemmatized.

    Returns:
        str: A string containing the lemmatized words, separated by spaces.

    Example:
        >>> lemmatizer_spacy("Os carros estão correndo rapidamente.")
        'carro correr rapidamente'

    Notes:
        - This function assumes that the `nlp` object (spaCy language model) is 
          already loaded and available in the global scope.
        - Stop words (e.g., "os", "estão") and punctuation are removed from the output.
        - The input text is converted to lowercase before processing.
    """
    str_text = str(text)
    doc = nlp(str_text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])


def remove_punctuation_and_accents(text: str) -> str:
    """
    Removes punctuation and accents from the input text.

    This function removes all punctuation and converts accented characters 
    (e.g., "ã", "ç") to their unaccented equivalents (e.g., "a", "c").

    Args:
        text (str): The input text to be processed.

    Returns:
        str: A cleaned string without punctuation or accents.

    Example:
        >>> remove_punctuation_and_accents("pão, maçã!")
        'pao maca'
    """
    # Normalize text to decompose accents
    normalized_text = unicodedata.normalize('NFD', text)
    # Remove accents by filtering out combining characters
    no_accents = ''.join(char for char in normalized_text if unicodedata.category(char) != 'Mn')
    # Remove punctuation
    cleaned_text = no_accents.translate(str.maketrans('', '', string.punctuation))
    return cleaned_text


def process_text(text: str) -> str:
    text = lemmatizer_spacy(text)
    return remove_punctuation_and_accents(text)

In [10]:
df["titulo_lemmatized"] = df["titulo"].apply(process_text)
df["resumo_lemmatized"] = df["resumo"].apply(process_text)

In [11]:
# Compairing the original and lemmatized text
df[["titulo", "titulo_lemmatized"]].head(10)

,titulo,titulo_lemmatized
0,TRATAMENTO DE COLISÕES EM UPLINK,tratamento colisao uplink
1,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,alojamento vela ignicao protecao anticorrosivo...
2,ANTICORPOS QUE RECONHECEM TAU,anticorpo reconhecer tau
3,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,aquecedor ar lenha dupla exaustao utilizar amb...
4,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,biblioteca celula unico nucleo unico alto rend...
5,CAPACIDADE DE MONITORAMENTO DE CANAL DE CONTRO...,capacidade monitoramento canal controle comuni...
6,"CÉLULA, CONSTRUÇÃO DE EXPRESSÃO, PLANTA OU PAR...",celula construcao expressao planta planta conj...
7,COMPARTILHAMENTO DE TEMPO DE OCUPAÇÃO DE CANAL...,compartilhamento ocupacao canal cot uplink ul ...
8,COMPOSIÇÃO FARMACÊUTICA BASEADA EM COMPOSTOS 1...,composicao farmaceutico basear composto 13 4ox...
9,CONFIGURAÇÃO DE TURBOMÁQUINA COMPRESSOR-EXPANS...,configuracao turbomaquina compressorexpansor m...


In [12]:
# Compairing the original and lemmatized text
df[["resumo", "resumo_lemmatized"]].head(10)

,resumo,resumo_lemmatized
0,"A presente invenção se refere a métodos, sis...",presente invencao referir metodo sistema di...
1,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,alojamento vela ignicao protecao anticorros...
2,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,anticorpo reconhecer tau invencao fornecer ...
3,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,aquecedor ar lenha dupla exaustao utilizar ...
4,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,biblioteca celula unico nucleo unico alto r...
5,CAPACIDADE DE MONITORAMENTO DE CANAL DE CONT...,capacidade monitoramento canal controle com...
6,"CÉLULA, CONSTRUÇÃO DE EXPRESSÃO, PLANTA OU P...",celula construcao expressao planta planta c...
7,COMPARTILHAMENTO DE TEMPO DE OCUPAÇÃO DE CAN...,compartilhamento ocupacao canal cot uplink ...
8,COMPOSIÇÃO FARMACÊUTICA BASEADA EM COMPOSTOS...,composicao farmaceutico basear composto 13 ...
9,CONFIGURAÇÃO DE TURBOMÁQUINA COMPRESSOREXPAN...,configuracao turbomaquina compressorexpanso...


In [13]:
df.to_csv("../../data/processed/patentes_inpi_lemmatized.csv", index=False)

In [14]:
df_match.head()

,TIPO DE EFEITO,TAREFA,OBJETO,EFEITO FÍSICO,SINONIMO 1 EFEITO FISICO,SINONIMO 2 EFEITO FISICO,PT Link,PT Description,Link Wiki (English)
0,Aplicação,Apertar,Sólido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array
1,Aplicação,Apertar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array
2,Aplicação,Concentrar,Campo,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array
3,Aplicação,Concentrar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array
4,Aplicação,Depositar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array


In [15]:
df_match["TAREFA_lemmatized"] = df_match["TAREFA"].apply(process_text)
df_match.head()

,TIPO DE EFEITO,TAREFA,OBJETO,EFEITO FÍSICO,SINONIMO 1 EFEITO FISICO,SINONIMO 2 EFEITO FISICO,PT Link,PT Description,Link Wiki (English),TAREFA_lemmatized
0,Aplicação,Apertar,Sólido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
1,Aplicação,Apertar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
2,Aplicação,Concentrar,Campo,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
3,Aplicação,Concentrar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
4,Aplicação,Depositar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,depositar


In [16]:
df_match.to_csv("../../data/processed/base_efeitos_físicos_publicada_lemmatized.csv", index=False)